In [0]:
# some libraries
import pandas as pd
import numpy as np

In [2]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [3]:
from google.colab import files
uploaded = files.upload()

Saving chn_series.csv to chn_series.csv


In [9]:
# to read the dataframe
import io
df = pd.read_csv(io.StringIO(uploaded['chn_series.csv'].decode('utf-8')))
df

,Date,CHN_TM_TAX_MRCH_BR_ZS,CHN_TM_TAX_TCOM_BR_ZS,CHN_FM_LBL_BMNY_ZG,CHN_FM_LBL_BMNY_GD_ZS,CHN_FM_LBL_BMNY_CN,CHN_IC_BUS_DISC_XQ,CHN_FM_LBL_BMNY_IR_ZS,CHN_IQ_WEF_CUST_XQ,CHN_NE_GDI_STKB_KN,CHN_AG_YLD_CREL_KG,CHN_AG_PRD_CREL_MT,CHN_NE_GDI_STKB_CN,CHN_NE_GDI_STKB_CD,CHN_BN_RES_INCL_CD,CHN_NV_MNF_CHEM_ZS_UN,CHN_SE_PRM_UNER_FE,CHN_SE_PRM_UNER,CHN_SE_PRM_UNER_MA,CHN_FM_AST_CGOV_ZG_M3,CHN_FS_AST_CGOV_GD_ZS,CHN_FM_AST_PRVT_ZG_M3,CHN_EN_ATM_CO2E_PP_GD_KD,CHN_EN_ATM_CO2E_PP_GD,CHN_EN_ATM_CO2E_KD_GD,CHN_EN_ATM_CO2E_KT,CHN_EN_CO2_ETOT_ZS,CHN_EN_ATM_CO2E_PC,CHN_EN_ATM_CO2E_GF_KT,CHN_EN_CO2_ETOT_MT,CHN_EN_ATM_CO2E_GF_ZS,CHN_EN_ATM_CO2E_LF_ZS,CHN_EN_ATM_CO2E_LF_KT,CHN_EN_CO2_OTHX_ZS,CHN_EN_CO2_MANF_ZS,CHN_EN_CO2_MANF_MT,CHN_EN_CO2_BLDG_MT,CHN_EN_CO2_OTHX_MT,CHN_EN_CO2_BLDG_ZS,CHN_EN_ATM_CO2E_SF_ZS,...,CHN_EA_PRD_AGRI_KD,CHN_TM_VAL_AGRI_ZS_UN,CHN_NV_AGR_TOTL_ZS,CHN_NV_AGR_TOTL_KN,CHN_NV_AGR_TOTL_KD_ZG,CHN_NV_AGR_TOTL_KD,CHN_IS_AIR_GOOD_MT_K1,CHN_NV_AGR_TOTL_CN,CHN_NV_AGR_TOTL_CD,CHN_IS_AIR_DPRT,CHN_IS_AIR_PSGR,CHN_EG_USE_COMM_CL_ZS,CHN_ER_H2O_FWAG_ZS,CHN_ER_H2O_FWIN_ZS,CHN_ER_H2O_FWDM_ZS,CHN_ER_H2O_FWTL_ZS,CHN_ER_H2O_FWTL_K3,CHN_AG_LND_ARBL_ZS,CHN_AG_LND_ARBL_HA_PC,CHN_AG_LND_ARBL_HA,CHN_MS_MIL_TOTL_TF_ZS,CHN_MS_MIL_TOTL_P1,CHN_MS_MIL_XPRT_KD,CHN_MS_MIL_MPRT_KD,CHN_AG_LND_PRCP_MM,CHN_IC_TAX_METG,CHN_FB_ATM_TOTL_P5,CHN_FB_BNK_CAPA_ZS,CHN_IC_CUS_DURS_EX,CHN_FB_AST_NPER_ZS,CHN_TM_TAX_MRCH_BC_ZS,CHN_VC_BTL_DETH,CHN_TM_TAX_TCOM_BC_ZS,CHN_EN_BIR_THRD_NO,CHN_TM_TAX_MANF_BC_ZS,CHN_SH_STA_BRTC_ZS,CHN_FB_CBK_BRWR_P3,CHN_SP_DYN_CBRT_IN,CHN_TM_TAX_MANF_BR_ZS,CPI
0,2001-12-31,10.02,11.32,15.042414,141.085625,1.564119e+13,NaN,8.587306,NaN,0.0,4800.3,396483872.0,2.314900e+11,2.796752e+10,4.744654e+10,11.525418,NaN,NaN,NaN,3.780782,9.936478,7.770360,0.688912,0.849985,1.438972,3.487566e+06,51.225522,2.742121,67212.443,1647.27,1.927202,18.919783,659839.980,2.651186,29.242298,1136.22,281.27,72.10,8.563851,69.700274,...,793.558894,4.572290,13.983461,3.437704e+12,2.650690,4.011766e+11,4232.3120,1.550250e+12,1.872939e+11,840911.0,72660653.0,2.298897,NaN,NaN,NaN,NaN,NaN,12.364652,0.091270,116082000.0,0.514400,3810000.0,5.150000e+08,2.850000e+09,NaN,NaN,NaN,4.100000,NaN,29.800000,100.0,NaN,100.0,NaN,100.0,97.3,NaN,13.38,9.64,35.0
1,2002-12-31,10.00,11.27,13.140436,145.390145,1.769651e+13,NaN,7.180891,NaN,0.0,4869.0,396676773.0,1.332900e+11,1.610366e+10,7.521693e+10,11.308560,NaN,NaN,NaN,1.480718,10.953165,13.437287,0.696924,0.846870,1.455708,3.850269e+06,52.624024,3.007083,54036.912,1823.68,1.403458,16.975975,653620.748,2.729820,28.172876,1144.13,284.95,77.07,8.108541,72.229894,...,814.098728,4.254246,13.301488,3.530122e+12,2.688362,4.119616e+11,5013.5050,1.619023e+12,1.956050e+11,932064.0,83671798.0,2.420432,NaN,NaN,NaN,19.544970,NaN,12.196144,0.089425,114500000.0,0.504444,3770000.0,5.260000e+08,2.891000e+09,645.0,NaN,NaN,NaN,NaN,26.000000,100.0,NaN,100.0,NaN,100.0,96.7,NaN,12.86,9.63,35.0
2,2003-12-31,10.00,11.27,19.239767,153.550935,2.110128e+13,NaN,6.125399,NaN,0.0,4871.1,374819700.0,1.872300e+11,2.262051e+10,1.057113e+11,10.945598,NaN,NaN,NaN,-0.086684,9.589809,16.773840,0.746891,0.889843,1.560076,4.540417e+06,53.758309,3.524074,60795.193,2149.79,1.338978,16.027033,727694.148,2.738475,27.719599,1327.85,305.15,87.94,7.611126,73.165033,...,833.175330,3.985964,12.348999,3.614027e+12,2.376852,4.217534e+11,5650.6300,1.697025e+12,2.050289e+11,946354.0,86040642.0,2.521048,NaN,NaN,NaN,NaN,NaN,11.897899,0.086697,111700000.0,0.497089,3750000.0,7.000000e+08,2.373000e+09,NaN,14.4,NaN,3.800000,6.64,20.400000,100.0,NaN,100.0,NaN,100.0,95.9,NaN,12.41,9.63,34.0
3,2004-12-31,10.00,11.27,14.886920,149.793555,2.424261e+13,NaN,4.701807,NaN,0.0,5186.8,411571792.0,3.750700e+11,4.531582e+10,1.898491e+11,11.123810,NaN,NaN,NaN,1.164242,9.660896,9.146658,0.781853,0.906569,1.633104,5.233539e+06,50.294652,4.037991,71579.840,2434.77,1.367714,16.046116,839779.670,2.669012,31.376397,1596.27,354.19,102.63,7.462184,73.342891,...,884.492804,4.034505,12.916648,3.835344e+12,6.123822,4.475808e+11

In [0]:
import datetime
def datetiem(x):
    date_time_obj = datetime.datetime.strptime(x, '%Y-%m-%d' )
    return date_time_obj

In [0]:
df["Date"] = df["Date"].apply(lambda x: datetiem(x))
df.index = df.Date # index as date because time is not a variable
df=df.drop(['Date'], axis=1)

In [12]:
# df0 = df.fillna(0)
# removes columns where more than 30% of data is missing
# rationale for 0.7, if we can train test split for 0.7/ 0.3 we can accept at least 70% filled data
# fills remaining missing values with the mean of the column   
df_mean = df.loc[:, pd.notnull(df).sum()>len(df)*.80].fillna(df.mean())
# df_mean = df.loc[:, pd.notnull(df).sum()>len(df)*.70].fillna(df.interpolate())
df_mean

,CHN_TM_TAX_MRCH_BR_ZS,CHN_TM_TAX_TCOM_BR_ZS,CHN_FM_LBL_BMNY_ZG,CHN_FM_LBL_BMNY_GD_ZS,CHN_FM_LBL_BMNY_CN,CHN_FM_LBL_BMNY_IR_ZS,CHN_NE_GDI_STKB_KN,CHN_AG_YLD_CREL_KG,CHN_AG_PRD_CREL_MT,CHN_NE_GDI_STKB_CN,CHN_NE_GDI_STKB_CD,CHN_BN_RES_INCL_CD,CHN_NV_MNF_CHEM_ZS_UN,CHN_FM_AST_CGOV_ZG_M3,CHN_FS_AST_CGOV_GD_ZS,CHN_FM_AST_PRVT_ZG_M3,CHN_EN_ATM_CO2E_PP_GD_KD,CHN_EN_ATM_CO2E_PP_GD,CHN_EN_ATM_CO2E_KD_GD,CHN_EN_ATM_CO2E_KT,CHN_EN_CO2_ETOT_ZS,CHN_EN_ATM_CO2E_PC,CHN_EN_ATM_CO2E_GF_KT,CHN_EN_ATM_CO2E_GF_ZS,CHN_EN_ATM_CO2E_LF_ZS,CHN_EN_ATM_CO2E_LF_KT,CHN_EN_CO2_OTHX_ZS,CHN_EN_CO2_MANF_ZS,CHN_EN_CO2_BLDG_ZS,CHN_EN_ATM_CO2E_SF_ZS,CHN_EN_CO2_TRAN_ZS,CHN_EN_ATM_CO2E_SF_KT,CHN_EN_ATM_CO2E_EG_ZS,CHN_EG_USE_CRNW_ZS,CHN_NY_GDP_COAL_RT_ZS,CHN_TX_VAL_SERV_CD_WT,CHN_TM_VAL_SERV_CD_WT,CHN_DT_NFL_PCBO_CD,CHN_BX_GSR_CMCP_ZS,CHN_BM_GSR_CMCP_ZS,...,CHN_NY_ADJ_NNAT_GN_ZS,CHN_NY_ADJ_DFOR_GN_ZS,CHN_NY_ADJ_NNAT_CD,CHN_NY_ADJ_DPEM_GN_ZS,CHN_NY_ADJ_DPEM_CD,CHN_SP_POP_DPND,CHN_SP_ADO_TFRT,CHN_SP_POP_DPND_OL,CHN_SP_POP_DPND_YG,CHN_AG_LND_AGRI_K2,CHN_AG_LND_AGRI_ZS,CHN_TX_VAL_AGRI_ZS_UN,CHN_EA_PRD_AGRI_KD,CHN_TM_VAL_AGRI_ZS_UN,CHN_NV_AGR_TOTL_ZS,CHN_NV_AGR_TOTL_KN,CHN_NV_AGR_TOTL_KD_ZG,CHN_NV_AGR_TOTL_KD,CHN_IS_AIR_GOOD_MT_K1,CHN_NV_AGR_TOTL_CN,CHN_NV_AGR_TOTL_CD,CHN_IS_AIR_DPRT,CHN_IS_AIR_PSGR,CHN_EG_USE_COMM_CL_ZS,CHN_AG_LND_ARBL_ZS,CHN_AG_LND_ARBL_HA_PC,CHN_AG_LND_ARBL_HA,CHN_MS_MIL_TOTL_TF_ZS,CHN_MS_MIL_TOTL_P1,CHN_MS_MIL_XPRT_KD,CHN_MS_MIL_MPRT_KD,CHN_FB_BNK_CAPA_ZS,CHN_FB_AST_NPER_ZS,CHN_TM_TAX_MRCH_BC_ZS,CHN_TM_TAX_TCOM_BC_ZS,CHN_TM_TAX_MANF_BC_ZS,CHN_SH_STA_BRTC_ZS,CHN_SP_DYN_CBRT_IN,CHN_TM_TAX_MANF_BR_ZS,CPI
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001-12-31,10.02,11.32,15.042414,141.085625,1.564119e+13,8.587306,0.0,4800.3,396483872.0,2.314900e+11,2.796752e+10,4.744654e+10,11.525418,3.780782,9.936478,7.770360,0.688912,0.849985,1.438972,3.487566e+06,51.225522,2.742121,67212.44300,1.927202,18.919783,6.598400e+05,2.651186,29.242298,8.563851,69.700274,8.317452,2.430843e+06,2.952290,15.888205,0.864466,8.967600e+10,3.922200e+10,-5.651338e+09,74.749470,28.980916,...,23.135111,0.0,3.053835e+11,0.486923,6.427382e+09,44.790923,8.7172,10.234984,34.555939,5199150.0,55.379541,0.863739,793.558894,4.572290,13.983461,3.437704e+12,2.650690,4.011766e+11,4232.3120,1.550250e+12,1.872939e+11,840911.0,72660653.0,2.298897,12.364652,0.091270,116082000.0,0.514400,3810000.0,5.150000e+08,2.850000e+09,4.100000,29.800000,100.0,100.0,100.0,97.300000,13.38,9.64,35.0
2002-12-31,10.00,11.27,13.140436,145.390145,1.769651e+13,7.180891,0.0,4869.0,396676773.0,1.332900e+11,1.610366e+10,7.521693e+10,11.308560,1.480718,10.953165,13.437287,0.696924,0.846870,1.455708,3.850269e+06,52.624024,3.007083,54036.91200,1.403458,16.975975,6.536207e+05,2.729820,28.172876,8.108541,72.229894,8.364739,2.781045e+06,3.053463,14.664065,0.262391,1.018030e+11,4.631434e+10,-3.395153e+09,74.193864,30.830658,...,24.564936,0.0,3.586481e+11,0.443083,6.469016e+09,43.091202,8.4200,10.350578,32.740624,5185330.0,55.232342,0.780541,814.098728,4.254246,13.301488,3.530122e+12,2.688362,4.119616e+11,5013.5050,1.619023e+12,1.956050e+11,932064.0,83671798.0,2.420432,12.196144,0.089425,114500000.0,0.504444,3770000.0,5.260000e+08,2.891000e+09,5.915007,26.000000,100.0,100.0,100.0,96.700000,12.86,9.63,35.0
2003-12-31,10.00,11.27,19.239767,153.550935,2.110128e+13,6.125399,0.0,4871.1,374819700.0,1.872300e+11,2.262051e+10,1.057113e+11,10.945598,-0.086684,9.589809,16.773840,0.746891,0.889843,1.560076,4.540417e+06,53.758309,3.524074,60795.19300,1.338978,16.027033,7.276941e+05,2.738475,27.719599,7.611126,73.165033,8.172491,3.321998e+06,3.150907,12.355613,0.312366,5.731979e+10,5.509253e+10,-2.932125e+09,55.263754,31.198607,...,27.758350,0.0,4.580128e+11,0.403868,6.663827e+09,41.192222,8.1580,10.446913,30.745309,5161330.0,54.976706,0.643643,833.175330,3.985964,12.348999,3.614027e+12,2.376852,4.217534e+11,5650.6300,1.697025e+12,2.050289e+11,946354.0,86040642.0,2.521048,11.897899,0.086697,111700000.0,

In [0]:
CPI_list = list(df_mean.CPI)
def pct_df(df):
    df.drop(["CPI"], axis=1)
    new_df = df.pct_change()
    new_df.replace(np.nan, 0)
    new_df = new_df.replace([np.inf, -np.inf], 0)
    new_df = new_df.dropna(how='all')
    new_df['CPI']=CPI_list[1:]
    new_df = new_df.fillna(df.mean())
    return new_df
def UpsampleByTimeIndex(df,bytime,method):
    upsampled = df.resample(bytime)
    interpolated = upsampled.interpolate(method=method)
    return interpolated
# Ignore the index and treat the values as equally space
# ‘nearest’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘spline’, ‘barycentric’, ‘polynomial’
# .agg aggregates/ groups by specified time frame
# D for days, M for months
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.resample.Resampler.aggregate.html 
# https://machinelearningmastery.com/resample-interpolate-time-series-data-python/

In [0]:
pct_df_mean = pct_df(df_mean)

In [0]:
#Ignore this code; helper functions for binarizing
# list_col_names = list(pct_df_mean.columns.values)
from itertools import repeat
def binarize(x):
    if x< 0:
        x = -1
    elif x>0:
        x=1
    else:
        x = 0
    return x
month_cpi = [x for item in CPI_list[1:len(CPI_list)-1] for x in repeat(item, 12)]
month_cpi.insert(len(a),CPI_list[len(CPI_list)-1])
def bin_action_cpi(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action(df):
    df = df.applymap(binarize)
    return df
def binarize10(x):
    if x<= 0:
        x = 0
    else:
        x = 1
    return x
def bin_action_cpi10(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize10)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action10(df):
    df = df.applymap(binarize10)
    return df
def binarize123(x):
    if x< 0:
        x = 1
    elif x>0:
        x=3
    else:
        x = 2
    return x
def bin_action_cpi123(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize123)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action123(df):
    df = df.applymap(binarize123)
    return df

NameError: ignored

In [0]:
# percentage change df and binarize 123
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'Y','linear')
# df_latest = bin_action_cpi123(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi = bin_action123(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df and binarize 0,1
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'M','linear')
# df_latest = bin_action_cpi10(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'M','linear')
# df_latest_wo_cpi = bin_action10(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df and binarize -1,0,1
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'Y','linear')
# df_latest = bin_action_cpi(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi = bin_action(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df
df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
df_latest = UpsampleByTimeIndex(pct_df_mean,'M','linear')
df_latest.index = list(range(len(df_latest)))
df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'M','linear')
df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# mean df
# df_mean_wo_cpi = df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(df_mean,'Y','linear')
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

In [16]:
df_latest_wo_cpi

,CHN_TM_TAX_MRCH_BR_ZS,CHN_TM_TAX_TCOM_BR_ZS,CHN_FM_LBL_BMNY_ZG,CHN_FM_LBL_BMNY_GD_ZS,CHN_FM_LBL_BMNY_CN,CHN_FM_LBL_BMNY_IR_ZS,CHN_NE_GDI_STKB_KN,CHN_AG_YLD_CREL_KG,CHN_AG_PRD_CREL_MT,CHN_NE_GDI_STKB_CN,CHN_NE_GDI_STKB_CD,CHN_BN_RES_INCL_CD,CHN_NV_MNF_CHEM_ZS_UN,CHN_FM_AST_CGOV_ZG_M3,CHN_FS_AST_CGOV_GD_ZS,CHN_FM_AST_PRVT_ZG_M3,CHN_EN_ATM_CO2E_PP_GD_KD,CHN_EN_ATM_CO2E_PP_GD,CHN_EN_ATM_CO2E_KD_GD,CHN_EN_ATM_CO2E_KT,CHN_EN_CO2_ETOT_ZS,CHN_EN_ATM_CO2E_PC,CHN_EN_ATM_CO2E_GF_KT,CHN_EN_ATM_CO2E_GF_ZS,CHN_EN_ATM_CO2E_LF_ZS,CHN_EN_ATM_CO2E_LF_KT,CHN_EN_CO2_OTHX_ZS,CHN_EN_CO2_MANF_ZS,CHN_EN_CO2_BLDG_ZS,CHN_EN_ATM_CO2E_SF_ZS,CHN_EN_CO2_TRAN_ZS,CHN_EN_ATM_CO2E_SF_KT,CHN_EN_ATM_CO2E_EG_ZS,CHN_EG_USE_CRNW_ZS,CHN_NY_GDP_COAL_RT_ZS,CHN_TX_VAL_SERV_CD_WT,CHN_TM_VAL_SERV_CD_WT,CHN_DT_NFL_PCBO_CD,CHN_BX_GSR_CMCP_ZS,CHN_BM_GSR_CMCP_ZS,...,CHN_NY_ADJ_DFOR_CD,CHN_NY_ADJ_NNAT_GN_ZS,CHN_NY_ADJ_DFOR_GN_ZS,CHN_NY_ADJ_NNAT_CD,CHN_NY_ADJ_DPEM_GN_ZS,CHN_NY_ADJ_DPEM_CD,CHN_SP_POP_DPND,CHN_SP_ADO_TFRT,CHN_SP_POP_DPND_OL,CHN_SP_POP_DPND_YG,CHN_AG_LND_AGRI_K2,CHN_AG_LND_AGRI_ZS,CHN_TX_VAL_AGRI_ZS_UN,CHN_EA_PRD_AGRI_KD,CHN_TM_VAL_AGRI_ZS_UN,CHN_NV_AGR_TOTL_ZS,CHN_NV_AGR_TOTL_KN,CHN_NV_AGR_TOTL_KD_ZG,CHN_NV_AGR_TOTL_KD,CHN_IS_AIR_GOOD_MT_K1,CHN_NV_AGR_TOTL_CN,CHN_NV_AGR_TOTL_CD,CHN_IS_AIR_DPRT,CHN_IS_AIR_PSGR,CHN_EG_USE_COMM_CL_ZS,CHN_AG_LND_ARBL_ZS,CHN_AG_LND_ARBL_HA_PC,CHN_AG_LND_ARBL_HA,CHN_MS_MIL_TOTL_TF_ZS,CHN_MS_MIL_TOTL_P1,CHN_MS_MIL_XPRT_KD,CHN_MS_MIL_MPRT_KD,CHN_FB_BNK_CAPA_ZS,CHN_FB_AST_NPER_ZS,CHN_TM_TAX_MRCH_BC_ZS,CHN_TM_TAX_TCOM_BC_ZS,CHN_TM_TAX_MANF_BC_ZS,CHN_SH_STA_BRTC_ZS,CHN_SP_DYN_CBRT_IN,CHN_TM_TAX_MANF_BR_ZS
0,-0.001996,-0.004417,-0.126441,0.030510,0.131404,-0.163778,0.0,0.014312,0.000487,-0.424208,-0.424201,0.585299,-1.881565e-02,-0.608357,0.102319,0.729300,0.011630,-0.003665,0.011630,0.103999,0.027301,0.096627,-0.196028,-0.271764,-0.102739,-0.009425,0.029660,-0.036571,-0.053167,0.036293,0.005685,0.144066,0.034269,-0.077047,-0.696470,0.135231,0.180826,-0.399230,-0.007433,0.063826,...,0.0,0.061803,0.0,0.174419,-0.090034,0.006478,-0.037948,-0.034094,0.011294,-0.052533,-0.002658,-0.002658,-0.096323,0.025883,-0.069559,-0.048770,0.026884,0.014212,0.026884,0.184578,0.044362,0.044375,0.108398,0.151542,0.052867,-0.013628,-0.020215,-0.013628,-0.019354,-0.010499,0.021359,0.014386,0.442685,-0.127517,0.0,0.0,0.0,-0.006166,-0.038864,-0.001037
1,-0.001830,-0.004049,-0.077224,0.032645,0.136487,-0.162379,0.0,0.013155,-0.004146,-0.355134,-0.355128,0.570309,-1.992237e-02,-0.645872,0.083419,0.689217,0.016636,0.000869,0.016636,0.110270,0.026822,0.102902,-0.169270,-0.252946,-0.098836,0.000804,0.027453,-0.034864,-0.053848,0.034347,0.003296,0.148270,0.034073,-0.083745,-0.622560,0.087549,0.181551,-0.377326,-0.028075,0.059502,...,0.0,0.067486,0.0,0.182972,-0.089907,0.008447,-0.038458,-0.033845,0.011128,-0.053234,-0.002822,-0.002822,-0.102912,0.025679,-0.069018,-0.050673,0.026624,0.003372,0.026624,0.179787,0.044680,0.044692,0.100642,0.141273,0.051925,-0.014530,-0.021073,-0.014530,-0.018956,-0.010066,0.047146,-0.001744,0.375997,-0.134839,0.0,0.0,0.0,-0.006342,-0.038541,-0.000951
2,-0.001663,-0.003681,-0.028007,0.034780,0.141570,-0.160980,0.0,0.011998,-0.008778,-0.286060,-0.286054,0.555319,-2.102908e-02,-0.683387,0.064520,0.649135,0.021641,0.005403,0.021641,0.116540,0.026343,0.109176,-0.142512,-0.234127,-0.094933,0.011033,0.025245,-0.033157,-0.054530,0.032402,0.000907,0.152474,0.033877,-0.090443,-0.548649,0.039867,0.182277,-0.355422,-0.048718,0.055178,...,0.0,0.073169,0.0,0.191525,-0.089779,0.010417,-0.038968,-0.033597,0.010963,-0.053934,-0.002987,-0.002986,-0.109501,0.025475,-0.068476,-0.052576,0.026364,-0.007469,0.026364,0.174996,0.044998,0.045009,0.092887,0.131004,0.050984,-0.015432,-0.021931,-0.015433,-0.018559,-0.009633,0.072932,-0.017874,0.309310,-0.142161,0.0,0.0,0.0,-0.006518,-0.038219,-0.000864
3,-0.001497,-0.003313,0.021211,0.036915,0.146653,-0.159580,0.0,0.010842,-0.013410,-0.216986,-0.216981,0.540329,-2.213579e-02,-0.720903,0.045621,0.609052,0.026647,0.009

In [17]:
# check for null, if false then no missing values
def null_check(dfr):
    return dfr.isnull().values.any()
# print(null_check(df0))
# print(null_check(df_mean))
print(null_check(df_latest_wo_cpi))

False


In [18]:
from sklearn.preprocessing import MinMaxScaler
sc= MinMaxScaler(feature_range=(0,1))

input_data = df_latest.values
test_data = df_latest_wo_cpi.values
input_data[:,0:len(input_data[0])-1] = sc.fit_transform(input_data[:,0:len(input_data[0])-1])
test_data[:,0:len(test_data[0])] = sc.fit_transform(test_data[:,:])
test_data

array([[0.39943477, 0.        , 0.24698996, ..., 0.3130883 , 0.        ,
        0.87512967],
       [0.41602811, 0.02969421, 0.29809305, ..., 0.30728131, 0.00288664,
        0.88553553],
       [0.43262144, 0.05938843, 0.34919614, ..., 0.30147431, 0.00577329,
        0.89594139],
       ...,
       [0.5985548 , 0.35633055, 0.28456818, ..., 0.08617954, 0.85512137,
        1.        ],
       [0.5985548 , 0.35633055, 0.25321678, ..., 0.04308977, 0.92756068,
        1.        ],
       [0.5985548 , 0.35633055, 0.22186537, ..., 0.        , 1.        ,
        1.        ]])

In [19]:
print(test_data.shape)
print(input_data.shape) # check data and shape, are there 22/ 229 rows?

(169, 648)
(169, 649)


In [20]:
df_latest.head()

,CHN_TM_TAX_MRCH_BR_ZS,CHN_TM_TAX_TCOM_BR_ZS,CHN_FM_LBL_BMNY_ZG,CHN_FM_LBL_BMNY_GD_ZS,CHN_FM_LBL_BMNY_CN,CHN_FM_LBL_BMNY_IR_ZS,CHN_NE_GDI_STKB_KN,CHN_AG_YLD_CREL_KG,CHN_AG_PRD_CREL_MT,CHN_NE_GDI_STKB_CN,CHN_NE_GDI_STKB_CD,CHN_BN_RES_INCL_CD,CHN_NV_MNF_CHEM_ZS_UN,CHN_FM_AST_CGOV_ZG_M3,CHN_FS_AST_CGOV_GD_ZS,CHN_FM_AST_PRVT_ZG_M3,CHN_EN_ATM_CO2E_PP_GD_KD,CHN_EN_ATM_CO2E_PP_GD,CHN_EN_ATM_CO2E_KD_GD,CHN_EN_ATM_CO2E_KT,CHN_EN_CO2_ETOT_ZS,CHN_EN_ATM_CO2E_PC,CHN_EN_ATM_CO2E_GF_KT,CHN_EN_ATM_CO2E_GF_ZS,CHN_EN_ATM_CO2E_LF_ZS,CHN_EN_ATM_CO2E_LF_KT,CHN_EN_CO2_OTHX_ZS,CHN_EN_CO2_MANF_ZS,CHN_EN_CO2_BLDG_ZS,CHN_EN_ATM_CO2E_SF_ZS,CHN_EN_CO2_TRAN_ZS,CHN_EN_ATM_CO2E_SF_KT,CHN_EN_ATM_CO2E_EG_ZS,CHN_EG_USE_CRNW_ZS,CHN_NY_GDP_COAL_RT_ZS,CHN_TX_VAL_SERV_CD_WT,CHN_TM_VAL_SERV_CD_WT,CHN_DT_NFL_PCBO_CD,CHN_BX_GSR_CMCP_ZS,CHN_BM_GSR_CMCP_ZS,...,CHN_NY_ADJ_NNAT_GN_ZS,CHN_NY_ADJ_DFOR_GN_ZS,CHN_NY_ADJ_NNAT_CD,CHN_NY_ADJ_DPEM_GN_ZS,CHN_NY_ADJ_DPEM_CD,CHN_SP_POP_DPND,CHN_SP_ADO_TFRT,CHN_SP_POP_DPND_OL,CHN_SP_POP_DPND_YG,CHN_AG_LND_AGRI_K2,CHN_AG_LND_AGRI_ZS,CHN_TX_VAL_AGRI_ZS_UN,CHN_EA_PRD_AGRI_KD,CHN_TM_VAL_AGRI_ZS_UN,CHN_NV_AGR_TOTL_ZS,CHN_NV_AGR_TOTL_KN,CHN_NV_AGR_TOTL_KD_ZG,CHN_NV_AGR_TOTL_KD,CHN_IS_AIR_GOOD_MT_K1,CHN_NV_AGR_TOTL_CN,CHN_NV_AGR_TOTL_CD,CHN_IS_AIR_DPRT,CHN_IS_AIR_PSGR,CHN_EG_USE_COMM_CL_ZS,CHN_AG_LND_ARBL_ZS,CHN_AG_LND_ARBL_HA_PC,CHN_AG_LND_ARBL_HA,CHN_MS_MIL_TOTL_TF_ZS,CHN_MS_MIL_TOTL_P1,CHN_MS_MIL_XPRT_KD,CHN_MS_MIL_MPRT_KD,CHN_FB_BNK_CAPA_ZS,CHN_FB_AST_NPER_ZS,CHN_TM_TAX_MRCH_BC_ZS,CHN_TM_TAX_TCOM_BC_ZS,CHN_TM_TAX_MANF_BC_ZS,CHN_SH_STA_BRTC_ZS,CHN_SP_DYN_CBRT_IN,CHN_TM_TAX_MANF_BR_ZS,CPI
0,0.399435,0.000000,0.246990,0.362921,0.054914,0.133782,0.0,0.391548,0.327069,0.052074,0.047353,0.955259,0.373268,0.168098,0.316656,0.676300,0.448924,0.322105,0.448924,0.856399,1.000000,0.848129,0.391109,0.208272,0.000000,0.619139,0.468778,0.006550,0.106964,1.000000,0.378786,0.903103,0.903171,0.077620,0.011249,0.495366,0.297293,0.618631,0.283106,0.126697,...,0.755528,0.0,0.558194,0.055223,0.031982,0.076178,0.049754,0.033192,0.121362,0.115561,0.115564,0.229969,0.019217,0.337372,0.345235,0.029706,0.148426,0.029706,0.436874,0.006146,0.183071,0.303227,0.338615,0.528007,0.114304,0.106931,0.114304,0.830844,0.858081,0.246117,0.530970,0.875407,0.509129,0.0,0.0,0.0,0.313088,0.000000,0.875130,35.000000
1,0.416028,0.029694,0.298093,0.372305,0.070358,0.136510,0.0,0.377611,0.299813,0.083040,0.076680,0.952074,0.360219,0.167642,0.293861,0.653405,0.462648,0.329636,0.462648,0.868366,0.994780,0.860784,0.416830,0.231877,0.014918,0.642974,0.459066,0.016614,0.104750,0.974277,0.368281,0.911177,0.900538,0.074459,0.020392,0.454086,0.298165,0.623299,0.267151,0.126108,...,0.775901,0.0,0.579780,0.056160,0.036328,0.069830,0.058289,0.031253,0.111248,0.110460,0.110462,0.210805,0.017615,0.339187,0.332081,0.027231,0.142517,0.027231,0.426884,0.007586,0.183979,0.277958,0.310397,0.526888,0.108575,0.101470,0.108574,0.832638,0.860044,0.260830,0.511195,0.802457,0.501449,0.0,0.0,0.0,0.307281,0.002887,0.885536,34.916667
2,0.432621,0.059388,0.349196,0.381688,0.085802,0.139238,0.0,0.363675,0.272557,0.114007,0.106006,0.948890,0.347171,0.167185,0.271066,0.630510,0.476371,0.337167,0.476371,0.880332,0.989559,0.873440,0.442550,0.255481,0.029837,0.666810,0.449355,0.026678,0.102536,0.948554,0.357777,0.919252,0.897905,0.071299,0.029535,0.412805,0.299037,0.627967,0.251195,0.125519,...,0.796273,0.0,0.601366,0.057097,0.040673,0.063482,0.066823,0.029313,0.101135,0.105359,0.105361,0.191641,0.016014,0.341003,0.318928,0.024755,0.136608,0.024755,0.416894,0.009026,0.184888,0.252689,0.282179,0.525769,0.102845,0.096010,0.102844,0.834431,0.862006,0.275543,0.491420,0.729506,0.493768,0.0,0.0,0.0,0.301474,0.005773,0.895941,34.833333
3,0.449215,0.089083,0.400299,0.391072,0.101245,0.141965,0.0,0.349738,0.245302,0.144974,0.135332,0.945706,0.334123,0.166729,0.248271,0.607615,0.490095,0.344698,0.490095,0.892299,0.984339,0.886096,0.468271,0.279085,0.044755,0.690645,0.439643,0.036741,0.100321,0.922831,0.347273,0.927327,0.8

In [21]:
n_steps = 3
df_test=test_data[-n_steps:]
CPI_test = df_latest['CPI'][-n_steps:]
print(CPI_test)
print(df_test.shape)
input_feature = input_data[:len(input_data)-n_steps] # to remove last 5 rows, follow the time step you set

166    39.50
167    39.75
168    40.00
Name: CPI, dtype: float64
(3, 648)


In [22]:
input_feature.shape # check shape 

(166, 649)

In [0]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [24]:
# choose a number of time steps
from numpy import array
# convert into input/output
X, y = split_sequences(input_feature, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])
# the array is (Total number of samples, time step, total number of dimensions)

(164, 3, 648) (164,)
[[0.39943477 0.         0.24698996 ... 0.3130883  0.         0.87512967]
 [0.41602811 0.02969421 0.29809305 ... 0.30728131 0.00288664 0.88553553]
 [0.43262144 0.05938843 0.34919614 ... 0.30147431 0.00577329 0.89594139]] 34.833333333333336
[[0.41602811 0.02969421 0.29809305 ... 0.30728131 0.00288664 0.88553553]
 [0.43262144 0.05938843 0.34919614 ... 0.30147431 0.00577329 0.89594139]
 [0.44921478 0.08908264 0.40029923 ... 0.29566732 0.00865993 0.90634725]] 34.75
[[0.43262144 0.05938843 0.34919614 ... 0.30147431 0.00577329 0.89594139]
 [0.44921478 0.08908264 0.40029923 ... 0.29566732 0.00865993 0.90634725]
 [0.46580811 0.11877685 0.45140232 ... 0.28986033 0.01154658 0.91675311]] 34.666666666666664
[[0.44921478 0.08908264 0.40029923 ... 0.29566732 0.00865993 0.90634725]
 [0.46580811 0.11877685 0.45140232 ... 0.28986033 0.01154658 0.91675311]
 [0.48240145 0.14847106 0.50250541 ... 0.28405333 0.01443322 0.92715897]] 34.583333333333336
[[0.46580811 0.11877685 0.45140232 .

In [0]:
# final model
# https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/ 
# refer to this if necessary
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, X.shape[2])))
model.add(Dense(15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# model.save_weights('my_model_weights.h5')

In [32]:
# fit model
model.fit(X, y, epochs=200, verbose=0)#, validation_split=0.2, batch_size=5)

In [0]:
# demonstrate prediction
x_input = df_test
x_input = x_input.reshape((1, n_steps,X.shape[2] ))
yhat = model.predict(x_input, verbose=0)

Percentage Error

In [34]:
print(yhat)
actual = CPI_test[len(df_latest)-1]
print(float((abs(actual-yhat)/actual)*100))

[[40.06353]]
0.15882492065429688


In [35]:
weight_tensors = model.layers[1].weights
weight_names = list(map(lambda x: x.name, weight_tensors))
print(weight_names)

['dense_3/kernel:0', 'dense_3/bias:0']


In [36]:
for layer in model.layers:
        if "LSTM" in str(layer):
            weightLSTM = layer.get_weights()
warr,uarr, barr = weightLSTM
print(warr.shape) # warr is a numpy array of weights for inputs
print(uarr.shape) # uarr is a numpy array of weights for hidden units
print(barr.shape) # barr is a numpy array of bias

(648, 200)
(50, 200)
(200,)


In [0]:
print(warr)
print(uarr) 
print(barr) 

[[-0.05996489  0.00334863 -0.07000631 ... -0.08312024 -0.05508691
   0.06340362]
 [ 0.03778417 -0.01975109  0.05824827 ... -0.0331271   0.00853069
  -0.05852728]
 [ 0.04641208  0.01114873 -0.03798164 ...  0.05747066 -0.04485274
   0.0443713 ]
 ...
 [ 0.05347044 -0.0572511   0.04036112 ... -0.02875088  0.05896998
  -0.07833659]
 [ 0.01239917  0.08323803 -0.01764604 ... -0.03914334 -0.06543626
  -0.01121462]
 [ 0.03294032  0.07181647  0.02700024 ... -0.00603995 -0.00806883
  -0.04849005]]
[[ 0.12589791 -0.07054115 -0.08861891 ... -0.03676573  0.05221381
  -0.15633503]
 [ 0.13306502 -0.12319936 -0.05175697 ...  0.02289279  0.04650595
  -0.06229677]
 [-0.13476533  0.02517678 -0.00739818 ... -0.06426033 -0.00743123
  -0.00591115]
 ...
 [ 0.01102621  0.00032036 -0.04485288 ...  0.12112253 -0.02007239
  -0.02193324]
 [-0.02039579  0.03089585 -0.00102132 ... -0.05920744 -0.02224196
   0.06074055]
 [-0.02116491 -0.0589334   0.04262791 ... -0.05624822 -0.01637447
  -0.04362038]]
[-2.03319232e-05

In [0]:
# change df to percentage changes
# play with the rank/ totaln
# interpolate lesser 
# time step

In [0]:
time step = 5, CPI, indicator data
interpolate by year, 22 data points
# 20.441774368286133
interpolate by month, 262 DP
# 1.0591048002243042
interpolate by day, 7920 DP
# 0.015229816548526287 

After performing percentage change and normalizing 
interpolate by year time step 7
# 0.6501209735870361
# 351430.6875
interpolate by year time step 5
# 0.5464288592338562
# 320606.21875
interpolate by year time step 3
# 1.6997615098953247
interpolate by month time step 7
# 1.4450266361236572
interpolate by month time step 5
# 0.217756450176239


*** champion ***
interpolate by month time step 3
# 0.4332916736602783 25 units
# 0.15020249783992767 50 units 0.24396678805351257
# 0.08212704956531525 75 units
# 0.025785423815250397 75 units dense 15 , 1
# 0.09989678114652634 75 units dense 25 , 1
***  winner  # 0.016958503052592278 75 units dense 35 , 1 
*** end ***


binarize -1,0,1
year time step = 7
# 63.047935485839844
year time step = 5
# 59.16572189331055
year time step = 3
# 82.13848114013672
month time step = 7
# 11.621817588806152
month time step = 5
# 11.747268676757812
month time step = 3
# 15.268765449523926


binarize 0,1
year time step = 7
# 1.3483071327209473
year time step = 5
# 2.037656784057617
year time step = 3
# 4.179565906524658
month time step = 7
# 0.44504478573799133
month time step = 5
# 4.9152421951293945
month time step = 3
# 3.9173173904418945

binarize 1,2,3
year time step = 7
# 5.180117607116699
year time step = 5
# 4.566781520843506
year time step = 3
# 1.8392828702926636
month time step = 7
# 3.652085065841675
month time step = 5
# 4.432069778442383
month time step = 3
# 0.7650640606880188